# Test the onnx runtime model against images (transform check)

In [6]:
import numpy as np
import onnx
import onnxruntime
import onnxruntime_extensions
import torch

from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

from torch.utils.data import Dataset
from torch import nn
from tqdm import tqdm
from datetime import date
from pathlib import Path
from onnxruntime_extensions import pnp, OrtPyFunction, get_library_path as _lib_path
import transformers
from transformers import logging, AutoTokenizer, AutoConfig, AutoModel
from transformers.onnx import FeaturesManager
from torchvision.transforms import transforms
from PIL import Image
logging.set_verbosity_error()

In [2]:
EXPORT_FOLDER = "../../onnx_models"
export_model_path = Path.joinpath(Path(EXPORT_FOLDER), f"EfficientNet-B3_Model.onnx")


In [3]:
onnx_model = onnx.load(export_model_path)
onnx.checker.check_model(onnx_model)

In [23]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # ResNet-18 expects images of size 224x224
    transforms.ToTensor(),
    # transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    # transforms.RandomRotation(degrees=5), # Added 3.20
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [42]:
img_path = Path.joinpath(Path(EXPORT_FOLDER), "3d2bae29-7504f8f4-4c6aaa8f-694ced3f-bb470179_min.jpg")

In [43]:
image = Image.open(img_path)

In [44]:
transformed_image = transform(image).unsqueeze(0)

In [45]:
transformed_image.numpy()[0][0][0][0]

-2.117904

In [49]:
transformed_image.type

<function Tensor.type>

In [47]:
transformed_image.shape

torch.Size([1, 3, 256, 256])

In [51]:
ort_session = onnxruntime.InferenceSession(export_model_path, providers=['CPUExecutionProvider'])
ort_session.run(None, {'input': transformed_image.numpy()})

[array([[-0.9833036 , -2.5338569 , -3.172876  , -1.8249557 , -0.15269934]],
       dtype=float32)]

In [36]:
image.getcolors()

[(199, (253, 253, 253)),
 (313, (251, 251, 251)),
 (283, (249, 249, 249)),
 (270, (247, 247, 247)),
 (238, (245, 245, 245)),
 (205, (243, 243, 243)),
 (215, (241, 241, 241)),
 (195, (239, 239, 239)),
 (241, (237, 237, 237)),
 (226, (235, 235, 235)),
 (234, (233, 233, 233)),
 (239, (231, 231, 231)),
 (210, (229, 229, 229)),
 (191, (227, 227, 227)),
 (187, (225, 225, 225)),
 (162, (223, 223, 223)),
 (172, (221, 221, 221)),
 (198, (219, 219, 219)),
 (215, (217, 217, 217)),
 (214, (215, 215, 215)),
 (210, (213, 213, 213)),
 (213, (211, 211, 211)),
 (212, (209, 209, 209)),
 (228, (207, 207, 207)),
 (183, (205, 205, 205)),
 (185, (203, 203, 203)),
 (224, (201, 201, 201)),
 (190, (199, 199, 199)),
 (196, (197, 197, 197)),
 (212, (195, 195, 195)),
 (232, (193, 193, 193)),
 (206, (191, 191, 191)),
 (222, (189, 189, 189)),
 (238, (187, 187, 187)),
 (202, (185, 185, 185)),
 (256, (183, 183, 183)),
 (243, (181, 181, 181)),
 (274, (179, 179, 179)),
 (272, (177, 177, 177)),
 (232, (175, 175, 175)),
